In [0]:
CREATE OR REPLACE VIEW vw_scontrini_and_budget AS
WITH cambio AS (
  SELECT v.valuta_id, cb.valore_cambio_eur
  FROM VALUTA v
  JOIN CAMBIO_BUDGET cb ON cb.valuta_id = v.valuta_id
)
SELECT
  'Vendite Reali' AS scenario,
  sctr.scontrino_id,
  sctr.data_scontrino,
  cli.cliente_id,
  cli.nome AS cliente_nome,
  cli.genere AS cliente_genere,
  cli.nazione AS cliente_nazione,
  pv.nome AS punto_vendita_nome,
  pv.paese AS punto_vendita_paese,
  soc.nome AS societa,
  prd.codice AS prodotto_cod,
  prd.nome AS prodotto,
  cat.nome AS prodotto_categoria,
  mar.nome AS prodotto_marchio,
  stg.nome AS prodotto_stagione,
  cal.mese_fiscale,
  cal.trimestre_fiscale,
  cal.semestre_fiscale,
  cal.anno_fiscale,
  val.codice AS valuta_scontrino,
  sctr.valuta_id,
  ds.quantita AS quantita_scontrino,
  ds.quantita * ds.prezzo_unitario_locale AS importo_locale_scontrino,
  NULL AS importo_locale_budget,
  -- Conversione in EUR usando la tabella di cambio
  (ds.quantita * ds.prezzo_unitario_locale) * cb.valore_cambio_eur AS importo_eur_scontrino,
  NULL AS importo_eur_budget
FROM DETTAGLIO_SCONTRINO ds
JOIN SCONTRINO sctr         ON ds.scontrino_id     = sctr.scontrino_id
JOIN CLIENTE cli            ON sctr.cliente_id     = cli.cliente_id
JOIN PUNTO_VENDITA pv       ON sctr.punto_vendita_id = pv.punto_vendita_id
JOIN SOCIETA soc            ON pv.societa_id       = soc.societa_id
JOIN VALUTA val             ON sctr.valuta_id      = val.valuta_id
JOIN cambio cb              ON val.valuta_id       = cb.valuta_id
JOIN PRODOTTO prd           ON ds.prodotto_id      = prd.prodotto_id
JOIN CATEGORIA cat          ON prd.categoria_id    = cat.categoria_id
JOIN MARCHIO mar            ON prd.marchio_id      = mar.marchio_id
JOIN STAGIONE stg           ON prd.stagione_id     = stg.stagione_id
JOIN CALENDARIO cal         ON sctr.data_scontrino = cal.data

UNION ALL

SELECT
  'Vendite Budget' AS scenario,
  NULL AS scontrino_id,
  bs.data AS data_scontrino,
  NULL AS cliente_id,
  NULL AS cliente_nome,
  NULL AS cliente_genere,
  NULL AS cliente_nazione,
  pv.nome AS punto_vendita_nome,
  pv.paese AS punto_vendita_paese,
  soc.nome AS societa,
  NULL AS prodotto_cod,
  NULL AS prodotto,
  NULL AS prodotto_categoria,
  NULL AS prodotto_marchio,
  NULL AS prodotto_stagione,
  cal.mese_fiscale,
  cal.trimestre_fiscale,
  cal.semestre_fiscale,
  cal.anno_fiscale,
  val.codice AS valuta_scontrino,
  val.valuta_id,
  NULL AS quantita_scontrino,
  NULL AS importo_locale_scontrino,
  bs.budget_vendite AS importo_locale_budget,
  NULL AS importo_eur_scontrino,
  bs.budget_vendite * cb.valore_cambio_eur AS importo_eur_budget
FROM BUDGET_SCONTRINO bs
JOIN PUNTO_VENDITA pv ON bs.punto_vendita_id = pv.punto_vendita_id
JOIN VALUTA val       ON bs.valuta_id        = val.valuta_id
JOIN cambio cb        ON val.valuta_id       = cb.valuta_id
JOIN CALENDARIO cal   ON bs.data             = cal.data
JOIN SOCIETA soc      ON pv.societa_id       = soc.societa_id;
